In [1]:
from google.cloud import bigquery

The following code cell fetches the taxi_trips table from the chicago_taxi_trips dataset. We also preview the first five rows of the table. You'll use the table to answer the questions below.

In [2]:
# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_taxi_trips" dataset
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,ef162bbd5b8e8dd6a079af0c548e46373f516be9,4b869d0c376995077b81f82d8549e484a62780a4a92277...,2013-02-22 10:15:00+00:00,2013-02-22 10:00:00+00:00,NaN,0.0,NaN,NaN,NaN,NaN,...,0.0,24.00,Credit Card,Park Ridge Taxi and Livery,NaN,NaN,None,NaN,NaN,None
1,9e75482e3d4773ee5ea5e23da3dec555c114e423,bee9782d8d7310bc1b902a8fe547baf0da4c95165bd81f...,2013-02-12 06:45:00+00:00,2013-02-12 06:45:00+00:00,NaN,0.0,NaN,NaN,NaN,NaN,...,0.0,47.34,Credit Card,Chicago Elite Cab Corp.,NaN,NaN,None,NaN,NaN,None
2,3e1c8cf338a4124cc23670d9a598e2e7d201d6a9,e428b282d868d83ee36a566f79e7e8926c9c3c631809cc...,2013-02-17 07:00:00+00:00,2013-02-17 07:00:00+00:00,NaN,0.0,NaN,NaN,NaN,NaN,...,0.0,7.35,Credit Card,Chicago Elite Cab Corp.,NaN,NaN,None,NaN,NaN,None
3,be32dc97303848671c115c66bc54b8585d553086,5c78e836fa4fd7ecadd6d3f7d9224f2af888dedeec3304...,2013-02-14 21:30:00+00:00,2013-02-14 21:30:00+00:00,NaN,0.0,NaN,NaN,NaN,NaN,...,0.0,18.81,Credit Card,Chicago Elite Cab Corp.,NaN,NaN,None,NaN,NaN,None
4,3fdc2b35dcc0d3e1ec1771f755330fc2da10d6e1,4c2ec0420ff3c82c7c0e76e2e6cf96208c2cff8b766c06...,2013-02-15 00:30:00+00:00,2013-02-15 00:30:00+00:00,0.0,0.0,NaN,NaN,NaN,NaN,...,0.0,5.00,Credit Card,Taxi Affiliation Services,NaN,NaN,None,NaN,NaN,None


# Exercises

## 1) How can you predict the demand for taxis?
Say you work for a taxi company, and you're interested in predicting the demand for taxis. Towards this goal, you'd like to create a plot that shows a rolling average of the daily number of taxi trips. Amend the (partial) query below to return a DataFrame with two columns:

* **trip_date** - contains one entry for each date from January 1, 2016, to March 31, 2016.
* **avg_num_trips** - shows the average number of daily trips, calculated over a window including the value for the current date, along with the values for the preceding 3 days and the following 3 days, as long as the days fit within the three-month time frame. For instance, when calculating the value in this column for January 3, 2016, the window will include the number of trips for the preceding 2 days, the current date, and the following 3 days.   

This query is partially completed for you, and you need only write the part that calculates the **avg_num_trips** column. Note that this query uses a common table expression (CTE); if you need to review how to use CTEs, you're encouraged to check out this tutorial in the Intro to SQL course.

In [3]:
avg_num_trips_query = """
                      WITH trips_by_day AS
                      (
                      SELECT DATE(trip_start_timestamp) AS trip_date,
                          COUNT(*) as num_trips
                      FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                      WHERE trip_start_timestamp >= '2016-01-01' AND trip_start_timestamp < '2016-04-01'
                      GROUP BY trip_date
                      )
                      SELECT trip_date,
                          AVG(num_trips) 
                          OVER (
                               ORDER BY trip_date
                               ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING
                               ) AS avg_num_trips
                      FROM trips_by_day
                      """

In [4]:
trip_number_query = """
                    SELECT pickup_community_area,
                        trip_start_timestamp,
                        trip_end_timestamp,
                        RANK()
                            OVER (
                                  PARTITION BY pickup_community_area
                                  ORDER BY trip_start_timestamp
                                 ) AS trip_number
                    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                    WHERE DATE(trip_start_timestamp) = '2013-10-03' 
                    """

trip_number_result = client.query(trip_number_query).result().to_dataframe()

In [5]:
trip_number_result

,pickup_community_area,trip_start_timestamp,trip_end_timestamp,trip_number
0,11.0,2013-10-03 00:00:00+00:00,2013-10-03 00:30:00+00:00,1
1,11.0,2013-10-03 00:00:00+00:00,2013-10-03 00:00:00+00:00,1
2,11.0,2013-10-03 00:30:00+00:00,2013-10-03 00:45:00+00:00,3
3,11.0,2013-10-03 00:45:00+00:00,2013-10-03 00:45:00+00:00,4
4,11.0,2013-10-03 00:45:00+00:00,2013-10-03 01:00:00+00:00,4
...,...,...,...,...
84263,15.0,2013-10-03 22:00:00+00:00,2013-10-03 22:00:00+00:00,50
84264,15.0,2013-10-03 22:00:00+00:00,2013-10-03 22:00:00+00:00,50
84265,15.0,2013-10-03 22:15:00+00:00,2013-10-03 22:30:00+00:00,52
84266,15.0,2013-10-03 23:30:00+00:00,2013-10-03 23:45:00+00:00,53


## 3) How much time elapses between trips?
The (partial) query in the code cell below shows, for each trip in the selected time frame, the corresponding `taxi_id`, `trip_start_timestamp`, and `trip_end_timestamp`. 

Your task in this exercise is to edit the query to include an additional `prev_break` column that shows the length of the break (in minutes) that the driver had before each trip started (this corresponds to the time between `trip_start_timestamp` of the current trip and `trip_end_timestamp` of the previous trip).  Partition the calculation by `taxi_id`, and order the results within each partition by `trip_start_timestamp`.

Some sample results are shown below, where all rows correspond to the same driver (or `taxi_id`).  Take the time now to make sure that the values in the `prev_break` column make sense to you!

![first_commands](https://storage.googleapis.com/kaggle-media/learn/images/qjvQzg8.png)

Note that the first trip of the day for each driver should have a value of **NaN** (not a number) in the `prev_break` column.

In [8]:
break_time_query = """
                   SELECT taxi_id,
                       trip_start_timestamp,
                       trip_end_timestamp,
                       TIMESTAMP_DIFF(
                           trip_start_timestamp, 
                           LAG(trip_end_timestamp, 1) OVER (PARTITION BY taxi_id ORDER BY trip_start_timestamp), 
                           MINUTE) as prev_break
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE DATE(trip_start_timestamp) = '2013-10-03' 
                   """
break_time_query_result = client.query(break_time_query).result().to_dataframe()

In [9]:
break_time_query_result

,taxi_id,trip_start_timestamp,trip_end_timestamp,prev_break
0,0439fb286bb3ee6c70870d2638535ca64734ed034c2d44...,2013-10-03 07:45:00+00:00,2013-10-03 08:00:00+00:00,225.0
1,1386732523ef7a6287b9004c7415180be4dea022dfff6a...,2013-10-03 15:45:00+00:00,2013-10-03 16:00:00+00:00,240.0
2,29d4cf448fc5dde30c6289bd89541cdcf63bc5433a7d80...,2013-10-03 07:45:00+00:00,2013-10-03 08:00:00+00:00,405.0
3,29d4cf448fc5dde30c6289bd89541cdcf63bc5433a7d80...,2013-10-03 19:00:00+00:00,2013-10-03 19:00:00+00:00,180.0
4,4241f559252855c582ba0e2de4f11663eb81835039b66f...,2013-10-03 18:15:00+00:00,2013-10-03 18:00:00+00:00,990.0
...,...,...,...,...
84263,bef8b8ce015170bb9aa8b1572ae3c5b704aafc09cf971c...,2013-10-03 21:30:00+00:00,2013-10-03 21:30:00+00:00,-15.0
84264,d626d13b670170414f2545181bb27d3f70edad7f0bffa4...,2013-10-03 20:15:00+00:00,2013-10-03 20:15:00+00:00,-15.0
84265,d626d13b670170414f2545181bb27d3f70edad7f0bffa4...,2013-10-03 23:45:00+00:00,2013-10-03 23:45:00+00:00,-15.0
84266,ddeba37f2a7773b1bbc516f2e46705c2f7a062ebb1ef5d...,2013-10-03 11:15:00+00:00,2013-10-03 11:15:00+00:00,-15.0


# Nested and Repeated Data

## Example
We'll work with the Google Analytics Sample dataset. It contains information tracking the behavior of visitors to the Google Merchandise store, an e-commerce website that sells Google branded items.

We begin by printing the first few rows of the ga_sessions_20170801 table. (We have hidden the corresponding code. To take a peek, click on the "Code" button below.) This table tracks visits to the website on August 1, 2017.

https://www.kaggle.com/datasets/bigquery/google-analytics-sample

In [10]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "google_analytics_sample" dataset
dataset_ref = client.dataset("google_analytics_sample", project="bigquery-public-data")

# Construct a reference to the "ga_sessions_20170801" table
table_ref = dataset_ref.table("ga_sessions_20170801")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


,visitorId,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,customDimensions,hits,fullVisitorId,userId,clientId,channelGrouping,socialEngagementType
0,NaN,1,1501591568,1501591568,20170801,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Europe', 'subContinent': 'South...",[],"[{'hitNumber': 1, 'time': 0, 'hour': 5, 'minut...",3418334011779872055,None,None,Organic Search,Not Socially Engaged
1,NaN,2,1501589647,1501589647,20170801,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': '/analytics/web/', 'campaign'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Asia', 'subContinent': 'Souther...","[{'index': 4, 'value': 'APAC'}]","[{'hitNumber': 1, 'time': 0, 'hour': 5, 'minut...",2474397855041322408,None,None,Referral,Not Socially Engaged
2,NaN,1,1501616621,1501616621,20170801,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': '/analytics/web/', 'campaign'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Europe', 'subContinent': 'North...","[{'index': 4, 'value': 'EMEA'}]","[{'hitNumber': 1, 'time': 0, 'hour': 12, 'minu...",5870462820713110108,None,None,Referral,Not Socially Engaged
3,NaN,1,1501601200,1501601200,20170801,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': '/analytics/web/', 'campaign'...","{'browser': 'Firefox', 'browserVersion': 'not ...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 8, 'minut...",9397809171349480379,None,None,Referral,Not Socially Engaged
4,NaN,1,1501615525,1501615525,20170801,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': '/analytics/web/', 'campaign'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 12, 'minu...",6089902943184578335,None,None,Referral,Not Socially Engaged


Description of eac hfield, refer to this data dictionary   
https://support.google.com/analytics/answer/3437719?hl=en   
The table has many nested fields, which you can verify by looking at either the data dictionary (hint: search for appearances of 'RECORD' on the page) or the table preview above.

In our first query against this table, we'll work with the "totals" and "device" columns.

In [12]:
print("SCHEMA field for the 'totals' column:\n")
print(table.schema[5])

print("\nSCHEMA field for the 'device' column:\n")
print(table.schema[7])

SCHEMA field for the 'totals' column:

SchemaField('totals', 'RECORD', 'NULLABLE', None, (SchemaField('visits', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('hits', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('pageviews', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('timeOnSite', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('bounces', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('transactions', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('transactionRevenue', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('newVisits', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('screenviews', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('uniqueScreenviews', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('timeOnScreen', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('totalTransactionRevenue', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('sessionQualityDim', 'INTEGER', 'NULLABLE', None, (), None)), None)

SCHEMA field for th

We refer to the "browser" field (which is nested in the "device" column) and the "transactions" field (which is nested inside the "totals" column) as device.browser and totals.transactions in the query below:

In [13]:
# Query to count the number of transactions per browser
query = """
        SELECT device.browser AS device_browser,
            SUM(totals.transactions) as total_transactions
        FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`
        GROUP BY device_browser
        ORDER BY total_transactions DESC
        """

# Run the query, and return a pandas DataFrame
result = client.query(query).result().to_dataframe()
result.head()

,device_browser,total_transactions
0,Chrome,41.0
1,Safari,3.0
2,Firefox,1.0
3,Edge,NaN
4,Coc Coc,NaN


By storing the information in the "device" and "totals" columns as STRUCTs (as opposed to separate tables), we avoid expensive JOINs. This increases performance and keeps us from having to worry about JOIN keys (and which tables have the exact data we need).

Now we'll work with the "hits" column as an example of data that is both nested and repeated. Since:

* "hits" is a STRUCT (contains nested data) and is repeated,
* "hitNumber", "page", and "type" are all nested inside the "hits" column, and
* "pagePath" is nested inside the "page" field,   

we can query these fields with the following syntax:

In [14]:
# Query to determine most popular landing point on the website
query = """
        SELECT hits.page.pagePath as path,
            COUNT(hits.page.pagePath) as counts
        FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`, 
            UNNEST(hits) as hits
        WHERE hits.type="PAGE" and hits.hitNumber=1
        GROUP BY path
        ORDER BY counts DESC
        """

# Run the query, and return a pandas DataFrame
result = client.query(query).result().to_dataframe()
result.head()

,path,counts
0,/home,1257
1,/google+redesign/shop+by+brand/youtube,587
2,/google+redesign/apparel/mens/mens+t+shirts,117
3,/signin.html,78
4,/basket.html,35


# Exercises

### 1) Who had the most commits in 2016?

GitHub is the most popular place to collaborate on software projects. A GitHub **repository** (or repo) is a collection of files associated with a specific project, and a GitHub **commit** is a change that a user has made to a repository.  We refer to the user as a **committer**.

The `sample_commits` table contains a small sample of GitHub commits, where each row corresponds to different commit.  The code cell below fetches the table and shows the first five rows of this table.

## 1) Who had the most commits in 2016?   

In [15]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "github_repos" dataset
dataset_ref = client.dataset("github_repos", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "sample_commits" table
table_ref = dataset_ref.table("sample_commits")

# API request - fetch the table
sample_commits_table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(sample_commits_table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


,commit,tree,parent,author,committer,subject,message,trailer,difference,difference_truncated,repo_name,encoding
0,afdba32e2a9ea729a9f9f280dbf6c718773c7ded,d77cca8a096e5320f3194d4a6ca1b4fef2dc9b99,[d65e55d4999b394e37ffe12543ecd2a17b7c44fc],"{'name': 'Jason Gunthorpe', 'email': 'a99b91d7...","{'name': 'Peter Huewe', 'email': '014f16385c5a...",tpm: Pull everything related to /dev/tpmX into...,tpm: Pull everything related to /dev/tpmX into...,"[{'key': 'Signed-off-by', 'value': 'Jason Gunt...","[{'old_mode': 33188.0, 'new_mode': 33188, 'old...",None,torvalds/linux,None
1,eb846d9f147455e4e5e1863bfb5e31974bb69b7c,443efbb146c7824508be817923bab04c2185810e,[3af6b35261182ff185db1f0fd271254147e2663e],"{'name': 'Hannes Reinecke', 'email': 'b0d1e9e4...","{'name': 'Christoph Hellwig', 'email': '923f77...",scsi: rename SERVICE_ACTION_IN to SERVICE_ACTI...,scsi: rename SERVICE_ACTION_IN to SERVICE_ACTI...,"[{'key': 'Signed-off-by', 'value': 'Hannes Rei...","[{'old_mode': 33188.0, 'new_mode': 33188, 'old...",None,torvalds/linux,None
2,f8798ccbefc0e4ef7438c080b7ba0410738c8cfa,9133440693c02314f1f6f95629b3594ce24ad0f8,[261e767628bb5971b9032439818237cc8511ea94],"{'name': 'Yong Zhang', 'email': '34add0fe16a1f...","{'name': 'Florian Tobias Schandinat', 'email':...",video: irq: Remove IRQF_DISABLED,video: irq: Remove IRQF_DISABLED\n\nSince comm...,"[{'key': 'Signed-off-by', 'value': 'Yong Zhang...","[{'old_mode': 33188.0, 'new_mode': 33188, 'old...",None,torvalds/linux,None
3,b83ae6d421435c6204150300f1c25bfbd39cd62b,99c6b661ab7de05c2fd49aa62624d2d6bf8abc69,[de1414a654e66b81b5348dbc5259ecf2fb61655e],"{'name': 'Christoph Hellwig', 'email': '923f77...","{'name': 'Jens Axboe', 'email': 'cd8c6775e60d6...",fs: remove mapping->backing_dev_info,fs: remove mapping->backing_dev_info\n\nNow th...,"[{'key': 'Signed-off-by', 'value': 'Christoph ...","[{'old_mode': 33188.0, 'new_mode': 33188, 'old...",None,torvalds/linux,None
4,aaabee8b7686dfe49f10289cb4b7a817b99e5dd9,7ccc6cf829a93d46daf484164a5466c91eca2efa,"[795e9364215dc98b1dea888ebae22383ecbbb92a, 2f2...","{'name': 'Luciano Coelho', 'email': 'd1ef58086...","{'name': 'Luciano Coelho', 'email': 'd1ef58086...",Merge branch 'wl12xx-next' into for-linville,Merge branch 'wl12xx-next' into for-linville\n...,"[{'key': 'Conflicts', 'value': '', 'email': No...","[{'old_mode': 33188.0, 'new_mode': 33188, 'old...",None,torvalds/linux,None


In [16]:
# Print information on all the columns in the table
sample_commits_table.schema

[SchemaField('commit', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('tree', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('parent', 'STRING', 'REPEATED', None, (), None),
 SchemaField('author', 'RECORD', 'NULLABLE', None, (SchemaField('name', 'STRING', 'NULLABLE', None, (), None), SchemaField('email', 'STRING', 'NULLABLE', None, (), None), SchemaField('time_sec', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('tz_offset', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('date', 'TIMESTAMP', 'NULLABLE', None, (), None)), None),
 SchemaField('committer', 'RECORD', 'NULLABLE', None, (SchemaField('name', 'STRING', 'NULLABLE', None, (), None), SchemaField('email', 'STRING', 'NULLABLE', None, (), None), SchemaField('time_sec', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('tz_offset', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('date', 'TIMESTAMP', 'NULLABLE', None, (), None)), None),
 SchemaField('subject', 'STRING', 'NULLABLE', None, (), None),
 SchemaFie

Write a query to find the individuals with the most commits in this table in 2016.  Your query should return a table with two columns:
- `committer_name` - contains the name of each individual with a commit (from 2016) in the table
- `num_commits` - shows the number of commits the individual has in the table (from 2016)

Sort the table, so that people with more commits appear first.

**NOTE**: You can find the name of each committer and the date of the commit under the "committer" column, in the "name" and "date" child fields, respectively.

In [18]:
# Write a query to find the answer
max_commits_query = """
                    SELECT committer.name AS committer_name, COUNT(*) AS num_commits
                    FROM `bigquery-public-data.github_repos.sample_commits`
                    WHERE committer.date >= '2016-01-01' AND committer.date < '2017-01-01'
                    GROUP BY committer_name
                    ORDER BY num_commits DESC

                    """
result = client.query(max_commits_query).result().to_dataframe()
result

,committer_name,num_commits
0,Greg Kroah-Hartman,3545
1,David S. Miller,3120
2,TensorFlower Gardener,2449
3,Linus Torvalds,2424
4,Benjamin Pasero,1127
...,...,...
748,naoyashiga,1
749,Jeff Shen,1
750,yanniks,1
751,Konstantin Lopuhin,1


### 2) Look at languages!

Now you will work with the `languages` table.  Run the code cell below to print the first few rows.

In [19]:
# Construct a reference to the "languages" table
table_ref = dataset_ref.table("languages")

# API request - fetch the table
languages_table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(languages_table, max_results=5).to_dataframe()

,repo_name,language
0,lemi136/puntovent,"[{'name': 'C', 'bytes': 80}]"
1,taxigps/nctool,"[{'name': 'C', 'bytes': 4461}]"
2,ahy1/strbuf,"[{'name': 'C', 'bytes': 5573}]"
3,nleiten/mod_rpaf-ng,"[{'name': 'C', 'bytes': 30330}]"
4,kmcallister/alameda,"[{'name': 'C', 'bytes': 17077}]"


Each row of the `languages` table corresponds to a different repository.  
- The "repo_name" column contains the name of the repository,
- the "name" field in the "language" column contains the programming languages that can be found in the repo, and 
- the "bytes" field in the "language" column has the size of the files (in bytes, for the corresponding language).

Run the following code cell to print the table schema.

In [20]:
# Print information on all the columns in the table
languages_table.schema

[SchemaField('repo_name', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('language', 'RECORD', 'REPEATED', None, (SchemaField('name', 'STRING', 'NULLABLE', None, (), None), SchemaField('bytes', 'INTEGER', 'NULLABLE', None, (), None)), None)]

Assume for the moment that you have access to a table called `sample_languages` that contains only a very small subset of the rows from the `languages` table: in fact, it contains only three rows!  This table is depicted in the image below.

![](https://storage.googleapis.com/kaggle-media/learn/images/qAb5lZ2.png)

How many rows are in the table returned by the query below?

![](https://storage.googleapis.com/kaggle-media/learn/images/Q5qYAtz.png)

Fill in your answer in the next code cell.

In [23]:
# Remember that the UNNEST() function essentially flattens the repeated data (which is then appended to the right side of the table) so that we have one element on each row.
num_rows = 6

### 3) What's the most popular programming language?

Write a query to leverage the information in the `languages` table to determine which programming languages appear in the most repositories.  The table returned by your query should have two columns:
- `language_name` - the name of the programming language
- `num_repos` - the number of repositories in the `languages` table that use the programming language

Sort the table so that languages that appear in more repos are shown first.

In [24]:
# The "language" column contains repeated data!

# Write a query to find the answer
pop_lang_query = """
                 SELECT l.name as language_name, COUNT(*) as num_repos
                 FROM `bigquery-public-data.github_repos.languages`,
                     UNNEST(language) AS l
                 GROUP BY language_name
                 ORDER BY num_repos DESC
                 """

result = client.query(pop_lang_query).result().to_dataframe()
result

,language_name,num_repos
0,JavaScript,1099966
1,CSS,807826
2,HTML,777433
3,Shell,640886
4,Python,550905
...,...,...
497,SVG,1
498,Mint,1
499,KiCad Schematic,1
500,Org,1


### 4) Which languages are used in the repository with the most languages?

For this question, you'll restrict your attention to the repository with name `'polyrabbit/polyglot'`.

Write a query that returns a table with one row for each language in this repository.  The table should have two columns:
- `name` - the name of the programming language
- `bytes` - the total number of bytes of that programming language

Sort the table by the `bytes` column so that programming languages that take up more space in the repo appear first.

In [27]:
# The "language" column contains repeated data! You'll need to use a WHERE clause and an ORDER BY clause.

all_langs_query = """
                  SELECT l.name, l.bytes
                  FROM `bigquery-public-data.github_repos.languages`,
                      UNNEST(language) as l
                  WHERE repo_name = 'polyrabbit/polyglot'
                  ORDER BY l.bytes DESC
                  """

result = client.query(all_langs_query).result().to_dataframe()
result

,name,bytes
0,Lasso,834726
1,C,819142
2,Mercury,709952
3,Objective-C,495392
4,Game Maker Language,298131
...,...,...
211,XC,82
212,Arduino,81
213,Nimrod,43
214,AutoHotkey,23


# Writing Eficient Queries

**Introduction**   
Sometimes it doesn't matter whether your query is efficient or not. For example, you might write a query you expect to run only once, and it might be working on a small dataset. In this case, anything that gives you the answer you need will do.

But what about queries that will be run many times, like a query that feeds data to a website? Those need to be efficient so you don't leave users waiting for your website to load.

Or what about queries on huge datasets? These can be slow and cost a business a lot of money if they are written poorly.

Most database systems have a query optimizer that attempts to interpret/execute your query in the most effective way possible. But several strategies can still yield huge savings in many cases.     

**Some useful functions**   
We will use two functions to compare the efficiency of different queries:

- show_amount_of_data_scanned() shows the amount of data the query uses.
- show_time_to_run() prints how long it takes for the query to execute.   


In [28]:
from google.cloud import bigquery
from time import time

client = bigquery.Client()

def show_amount_of_data_scanned(query):
    # dry_run lets us see how much data the query uses without running it
    dry_run_config = bigquery.QueryJobConfig(dry_run=True)
    query_job = client.query(query, job_config=dry_run_config)
    print('Data processed: {} GB'.format(round(query_job.total_bytes_processed / 10**9, 3)))
    
def show_time_to_run(query):
    time_config = bigquery.QueryJobConfig(use_query_cache=False)
    start = time()
    query_result = client.query(query, job_config=time_config).result()
    end = time()
    print('Time to run: {} seconds'.format(round(end-start, 3)))

Using Kaggle's public dataset BigQuery integration.


In [29]:
big_join_query = """
                 SELECT repo,
                     COUNT(DISTINCT c.committer.name) as num_committers,
                     COUNT(DISTINCT f.id) AS num_files
                 FROM `bigquery-public-data.github_repos.commits` AS c,
                     UNNEST(c.repo_name) AS repo
                 INNER JOIN `bigquery-public-data.github_repos.files` AS f
                     ON f.repo_name = repo
                 WHERE f.repo_name IN ( 'tensorflow/tensorflow', 'facebook/react', 'twbs/bootstrap', 'apple/swift', 'Microsoft/vscode', 'torvalds/linux')
                 GROUP BY repo
                 ORDER BY repo
                 """
show_time_to_run(big_join_query)

small_join_query = """
                   WITH commits AS
                   (
                   SELECT COUNT(DISTINCT committer.name) AS num_committers, repo
                   FROM `bigquery-public-data.github_repos.commits`,
                       UNNEST(repo_name) as repo
                   WHERE repo IN ( 'tensorflow/tensorflow', 'facebook/react', 'twbs/bootstrap', 'apple/swift', 'Microsoft/vscode', 'torvalds/linux')
                   GROUP BY repo
                   ),
                   files AS 
                   (
                   SELECT COUNT(DISTINCT id) AS num_files, repo_name as repo
                   FROM `bigquery-public-data.github_repos.files`
                   WHERE repo_name IN ( 'tensorflow/tensorflow', 'facebook/react', 'twbs/bootstrap', 'apple/swift', 'Microsoft/vscode', 'torvalds/linux')
                   GROUP BY repo
                   )
                   SELECT commits.repo, commits.num_committers, files.num_files
                   FROM commits 
                   INNER JOIN files
                       ON commits.repo = files.repo
                   ORDER BY repo
                   """

show_time_to_run(small_join_query)

Time to run: 16.037 seconds
Time to run: 3.988 seconds


# Exercise: Writing Efficient Queries

# Exercises

### 1) You work for Pet Costumes International.

You need to write three queries this afternoon. You have enough time to write working versions of all three, but only enough time to think about optimizing one of them.  Which of these queries is most worth optimizing?

1. A software engineer wrote an app for the shipping department, to see what items need to be shipped and which aisle of the warehouse to go to for those items. She wants you to write the query. It will involve data that is stored in an `orders` table, a `shipments` table and a `warehouseLocation` table. The employees in the shipping department will pull up this app on a tablet, hit refresh, and your query results will be shown in a nice interface so they can see what costumes to send where.


2. The CEO wants a list of all customer reviews and complaints… which are conveniently stored in a single `reviews` table. Some of the reviews are really long… because people love your pirate costumes for parrots, and they can’t stop writing about how cute they are.


3. Dog owners are getting more protective than ever. So your engineering department has made costumes with embedded GPS trackers and wireless communication devices. They send the costumes’ coordinates to your database once a second. You then have a website where owners can find the location of their dogs (or at least the costumes they have for those dogs). For this service to work, you need a query that shows the most recent location for all costumes owned by a given human. This will involve data in a `CostumeLocations` table as well as a `CostumeOwners` table.

So, which of these could benefit most from being written efficiently?  Set the value of the `query_to_optimize` variable below to one of `1`, `2`, or `3`.  (Your answer should have type **integer**.)

In [31]:
query_to_optimize = 3

Why 3: Because data is sent for each costume at each second, this is the query that is likely to involve the most data (by far). And it will be run on a recurring basis. So writing this well could pay off on a recurring basis.

Why not 1: This is the second most valuable query to optimize. It will be run on a recurring basis, and it involves merges, which is commonly a place where you can make your queries more efficient

Why not 2: This sounds like it will be run only one time. So, it probably doesn’t matter if it takes a few seconds extra or costs a few cents more to run that one time. Also, it doesn’t involve JOINs. While the data has text fields (the reviews), that is the data you need. So, you can’t leave these out of your select query to save computation.

### 2) Make it easier to find Mitzie! 

You have the following two tables:

![](https://storage.googleapis.com/kaggle-media/learn/images/E9jikOQ.png)

The `CostumeLocations` table shows timestamped GPS data for all of the pet costumes in the database, where `CostumeID` is a unique identifier for each costume.  

The `CostumeOwners` table shows who owns each costume, where the `OwnerID` column contains unique identifiers for each (human) owner.  Note that each owner can have more than one costume!  And, each costume can have more than one owner: this allows multiple individuals from the same household (all with their own, unique `OwnerID`) to access the locations of their pets' costumes.

Say you need to use these tables to get the current location of one pet in particular: Mitzie the Dog recently ran off chasing a squirrel, but thankfully she was last seen in her hot dog costume!

One of Mitzie's owners (with owner ID `MitzieOwnerID`) logs into your website to pull the last locations of every costume in his possession.  Currently, you get this information by running the following query:

```sql
WITH LocationsAndOwners AS 
(
SELECT * 
FROM CostumeOwners co INNER JOIN CostumeLocations cl
   ON co.CostumeID = cl.CostumeID
),
LastSeen AS
(
SELECT CostumeID, MAX(Timestamp)
FROM LocationsAndOwners
GROUP BY CostumeID
)
SELECT lo.CostumeID, Location 
FROM LocationsAndOwners lo INNER JOIN LastSeen ls 
	ON lo.Timestamp = ls.Timestamp AND lo.CostumeID = ls.CostumeID
WHERE OwnerID = MitzieOwnerID
```

Is there a way to make this faster or cheaper?

In [32]:
# Do you see any large merges in the query?

Solution: Yes. Working with the LocationsAndOwners table is very inefficient, because it’s a big table. There are a few options here, and which works best depends on database specifics. One likely improvement is    

WITH CurrentOwnersCostumes AS       
(    
SELECT CostumeID    
FROM CostumeOwners     
WHERE OwnerID = MitzieOwnerID     
),    
OwnersCostumesLocations AS     
(     
SELECT cc.CostumeID, Timestamp, Location     
FROM CurrentOwnersCostumes cc INNER JOIN CostumeLocations cl    
    ON cc.CostumeID = cl.CostumeID      
),    
LastSeen AS     
(    
SELECT CostumeID, MAX(Timestamp)     
FROM OwnersCostumesLocations      
GROUP BY CostumeID      
)
SELECT ocl.CostumeID, Location       
FROM OwnersCostumesLocations ocl INNER JOIN LastSeen ls     
    ON ocl.timestamp = ls.timestamp AND ocl.CostumeID = ls.costumeID        
    
    
Why is this better?

Instead of doing large merges and running calculations (like finding the last timestamp) for every costume, we discard the rows for other owners as the first step. So each subsequent step (like calculating the last timestamp) is working with something like 99.999% fewer rows than what was needed in the original query.   


Databases have something called “Query Planners” to optimize details of how a query executes even after you write it. Perhaps some query planner would figure out the ability to do this. But the original query as written would be very inefficient on large datasets.